# Getting the datas

In [ ]:
import requests
import os

data_list = []
# The path containing my datas
path_data = './data'
if not os.path.exists(path_data):
    os.makedirs(path_data)
# The name of the files
usagers_2022_file = 'usagers_2022.csv'
usagers_2022_link = 'https://www.data.gouv.fr/fr/datasets/r/62c20524-d442-46f5-bfd8-982c59763ec8'
vehicules_2022_file = 'vehicules_2022.csv'
vehicules_2022_link = 'https://www.data.gouv.fr/fr/datasets/r/c9742921-4427-41e5-81bc-f13af8bc31a0'
lieux_2022_file = 'lieux_2022.csv'
lieux_2022_link = 'https://www.data.gouv.fr/fr/datasets/r/a6ef711a-1f03-44cb-921a-0ce8ec975995'
caract_2022_file = 'caract_2022.csv'
caract_2022_link = 'https://www.data.gouv.fr/fr/datasets/r/5fc299c0-4598-4c29-b74c-6a67b0cc27e7'

data_list.append((usagers_2022_file,usagers_2022_link))
data_list.append((vehicules_2022_file,vehicules_2022_link))
data_list.append((lieux_2022_file,lieux_2022_link))
data_list.append((caract_2022_file,caract_2022_link))

usagers_2021_file = 'usagers_2021.csv'
usagers_2021_link = 'https://www.data.gouv.fr/fr/datasets/r/ba5a1956-7e82-41b7-a602-89d7dd484d7a'

vehicules_2021_file = 'vehicules_2021.csv'
vehicules_2021_link = 'https://www.data.gouv.fr/fr/datasets/r/0bb5953a-25d8-46f8-8c25-b5c2f5ba905e'

lieux_2021_file = 'lieux_2021.csv'
lieux_2021_link = 'https://www.data.gouv.fr/fr/datasets/r/8a4935aa-38cd-43af-bf10-0209d6d17434'

caract_2021_file = 'caract_2021.csv'
caract_2021_link = 'https://www.data.gouv.fr/fr/datasets/r/85cfdc0c-23e4-4674-9bcd-79a970d7269b'

data_list.append((usagers_2021_file,usagers_2021_link))
data_list.append((vehicules_2021_file,vehicules_2021_link))
data_list.append((lieux_2021_file,lieux_2021_link))
data_list.append((caract_2021_file,caract_2021_link))

for filename,link in data_list:
    if os.path.exists(os.path.join(path_data, filename)):
        print('The file %s already exists.' % os.path.join(path_data, filename))
    else:
        r = requests.get(link)
        with open(os.path.join(path_data, filename), 'wb') as f:
            f.write(r.content)
        print(f'Downloaded file {os.path.join(path_data, filename)}.')

# loading library and spark session 

In [ ]:
import numpy as np
import pandas as pd
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)   
# Set the compute.max_rows option to a larger value
ps.config.set_option('compute.max_rows', None)  


from pyspark.sql import SparkSession
from pyspark import SparkConf
spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
conf = SparkConf().setAppName("Data cleaning")


# Cleaning usager 2021

In [ ]:
file_path = os.path.join(path_data, usagers_2021_file)
sdf = ps.read_csv(file_path, sep=';')

In [ ]:
sdf.columns

In [ ]:
sdf.info()

In [ ]:
sdf.describe()

In [ ]:
sdf.head(2)

In [ ]:
type(sdf.loc[0, 'id_usager'])

In [ ]:
type(sdf.loc[0, 'id_vehicule'])

We can already see some string type for id_usager and id_vehicule that should be interger

In [ ]:
sdf['sexe'].unique()

In [ ]:
sdf['place'].unique()

In [ ]:
sdf['grav'].unique()

 The following columns : sexe, place and grav should not have -1 as values so we can think of seeing how many bad values we have and decide if we should delete the lines with thoses values

In [ ]:
(sdf['sexe'] == -1).sum() ,(sdf['place'] == -1).sum(),(sdf['grav'] == -1).sum() 

Now let's delete the rows with bad values and check again and we drop the useless column num_veh because xe have id_vehicule

In [ ]:
def delete_incorrect_values_usager(sdf):
    sdf = sdf.loc[(sdf['sexe'] != -1) & (sdf['place'] != -1) & (sdf['grav'] != -1)]
    sdf = sdf.drop(columns=['num_veh'])
    return sdf
sdf = delete_incorrect_values_usager(sdf)
# sdf = sdf.loc[(sdf['sexe'] != -1) & (sdf['place'] != -1) & (sdf['grav'] != -1)]
(sdf['sexe'] == -1).sum() ,(sdf['place'] == -1).sum(),(sdf['grav'] == -1).sum() 

It's time to attack column with Null values

In [ ]:
spam = sdf.isnull().sum()
spam

The column an_nais has 3067 missing values so we can also delete 3067 rows because we will need the column to show some datas

In [ ]:
sdf = sdf.dropna()
spam = sdf.isnull().sum()
spam

In [ ]:
sdf.info()

## Lets checkout categorical variables and all types
### Columns that should be categoricals:
- place
- catu
- grav
- sexe
- trajet
- secu1
- secu2
- secu3
- locp
- actp
- etatp
### Other types
- id_usager, id_vehicules, Num_Acc and num_veh are becoming string. We tried to spilt the space of every row in the id_usager and id_vehicule column but apprently those type of operations are impossible with pands on spark. It's not allowed
- column should be an Integer
- we are adding a column annee for the year of the file

In [ ]:
sdf['annee'] = 2021
sdf.head()

In [ ]:
usager_dtypes = {
    'Num_Acc': str,
    'id_usager': str,
    'id_vehicule': str,
    'place': np.int32,
    'catu': np.int8,
    'grav': np.int16,
    'sexe': np.int8,
    'an_nais': np.int64,
    'trajet': np.int32,
    'secu1': np.int32,
    'secu2': np.int32,
    'secu3': np.int32,
    'locp': np.int32,
    'actp': np.int32,
    'etatp': np.int32,
    'annee': np.int64,
}
sdf = sdf.astype(usager_dtypes)

In [ ]:
def clean_usager_df(sdf,year):
    sdf = delete_incorrect_values_usager(sdf)
    sdf = sdf.dropna()
    sdf['annee'] = year
    sdf = sdf.astype(usager_dtypes)
    return sdf
    
    
    

# Let's clean usager 2022 

In [ ]:
file_path2 = os.path.join(path_data, usagers_2022_file)
sdf2 = ps.read_csv(file_path2, sep=';')
sdf2 = clean_usager_df(sdf2,2022)
# sdf2 = sdf2.loc[(sdf2['sexe'] != -1) & (sdf2['place'] != -1) & (sdf2['grav'] != -1)]
# sdf2 = sdf2.dropna()
# sdf2['annee'] = 2022
# sdf2 = sdf2.astype(usager_dtypes)





Now let's concat the datas and save to parquet

In [ ]:
usager_df = ps.concat([sdf, sdf2])

In [ ]:
usager_df = usager_df.astype(usager_dtypes)
usager_df.info()

In [ ]:
usager_df.to_parquet(os.path.join("usager_cleaned.parquet"))

# Cleaning 2021 vehicules csv

In [ ]:
file_path3 = os.path.join(path_data, vehicules_2021_file)
sdf3 = ps.read_csv(file_path3, sep=';')

In [ ]:
sdf3.info()

The columns occutc will disappear cause it has a lot of null values, we also dont need the column num_veh because we have id_vehicule already

In [ ]:
sdf3.head()

In [ ]:
sdf3['catv'].unique()

catv column should not have -1 values so we delete the rows with that value

In [ ]:
def delete_incorrect_values_vehicules(sdf):
    sdf = sdf.loc[(sdf['catv'] != -1)]
    sdf = sdf.drop(columns=['occutc'])
    sdf = sdf.drop(columns=['num_veh'])
    return sdf
sdf3 = delete_incorrect_values_vehicules(sdf3)

In [ ]:
(sdf3['catv'] == -1).sum() 


In [ ]:
spam = sdf3.isnull().sum()
spam

Addind the year

In [ ]:
sdf3['annee'] = 2021


In [ ]:
sdf3.info()

Let's change the types

In [ ]:
vehicule_dtypes = {
    'Num_Acc': str,
    'id_vehicule': str,
    'senc': np.int32,
    'catv': np.int32,
    'obs': np.int32,
    'obsm': np.int32,
    'choc': np.int32,
    'manv': np.int32,
    'motor': np.int32,
    'annee': np.int64,
    
}
sdf3 = sdf3.astype(vehicule_dtypes)

In [ ]:
sdf3.info()

In [ ]:
def clean_vehicule_df(sdf,year):
    sdf = delete_incorrect_values_vehicules(sdf)
    sdf['annee'] = year
    sdf = sdf.astype(vehicule_dtypes)
    return sdf

# Let's clean 2022 vehicules data

In [ ]:
file_path4 = os.path.join(path_data, vehicules_2022_file)
sdf4 = ps.read_csv(file_path4, sep=';')
sdf4 = clean_vehicule_df(sdf4,2022)

In [ ]:
sdf4.head()

Now let's concat the datas and save to parquet

In [ ]:
vehicule_df = ps.concat([sdf3, sdf4])
vehicule_df = vehicule_df.astype(vehicule_dtypes)
# vehicule_df.info()

In [ ]:
vehicule_df.to_parquet(os.path.join("vehicule_cleaned.parquet"))


# Data cleaning: caract_2021

bilan caract_2021:
- type des colonnes 
- mise en date time de an 
- drop de la colonne commune (com)
- mise en float de lat et long
    

In [ ]:
file_path = os.path.join(path_data, caract_2021_file)
sdf_cara_21 = ps.read_csv(file_path, sep=';')
sdf_cara_21.columns

In [ ]:
sdf_cara_21.info()


In [ ]:
sdf_cara_21.describe()


In [ ]:
sdf_cara_21.head()

find out for null values 

In [ ]:
sdf_cara_21.isnull().sum()

In [ ]:
sdf_cara_21['lat'].isnull().sum()

we look for distinct values for the columns

In [ ]:
sdf_cara_21['dep'].unique()

In [ ]:
sdf_cara_21['lat'].unique()

In [ ]:
sdf_cara_21.isna().sum()

In [ ]:
(sdf_cara_21['int']==-1).sum()  # count the number of -1 in each column

all values for the columns are ok , they all have distinct values

In [ ]:
sdf_cara_21.columns

#columns data types 

** categotical columns :
- lum
- agg
- int
- atm
- col
- dep
- com
- lat
- long
- 
**numerical columns :
- jour
- mois

  
** continuos columns to be decided later:
- adr 
- Num_Acc
- lat
- long
  
** date columns:
- hrmn
- an


lets build the dictionary of the columns

In [ ]:
caract_types ={
    'lum': np.int16,
    'agg': np.int16,
    'int': np.int16,
    'atm': np.int16,
    'col': np.int16,
    'dep': np.int16,
    'Num_Acc': str,
    'jour': np.int64,
    'mois': np.int64,
    'adr' : str
}

In [ ]:

file_path = os.path.join(path_data, caract_2021_file)
cleaned_caract_21 = sdf_cara_21.astype(caract_types)


In [ ]:
cleaned_caract_21.info()


In [ ]:
cleaned_caract_21.head()

In [ ]:

cleaned_caract_21['lat'] = cleaned_caract_21['lat'].astype(float)
cleaned_caract_21['long'] = cleaned_caract_21['long'].astype(float)

In [ ]:
cleaned_caract_21.info()

In [ ]:
#drop the columns com
cleaned_caract_21 = cleaned_caract_21.drop(columns=['com'])

In [ ]:
cleaned_caract_21.info()

In [ ]:
# cleaned_caract_21['an'] = ps.to_datetime(cleaned_caract_21['an'], format='%Y')

In [ ]:
# takes a dataframe and a year as input and returns a cleaned dataframe
def clean_caract_df(sdf,year):
    #type conversion
    sdf = sdf.astype(caract_types)
    
    #drop the columns com
    sdf = sdf.drop(columns=['com'])
    
    # replace ',' by '.' and convert to float
    sdf['lat'] = sdf['lat'].astype(float)
    sdf['long'] = sdf['long'].astype(float)
    
    # convert the column 'an' to datetime
    # sdf['an'] = ps.to_datetime(sdf['an'], format='%Y')
    
    return sdf

saving the cleaned data in parquet format

In [ ]:
#cleaned_caract_21.to_parquet('caract_2021.parquet')

# Data cleaning : du fichier caract_2022.csv

In [ ]:
file_path = os.path.join(path_data, caract_2022_file)
sdf_cara_22 = ps.read_csv(file_path, sep=';')

In [ ]:
sdf_cara_22.info()

change the column name AccidentId to Num_Acc

In [ ]:

sdf_cara_22 = sdf_cara_22.rename(columns={'Accident_Id': 'Num_Acc'})

In [ ]:

cleaned_cara_22 = clean_caract_df(sdf_cara_22,2022)



In [ ]:
cleaned_cara_22.info()

In [ ]:
#cleaned_cara_22.to_parquet('caract_2022.parquet')

# concanate the two dataframes caract_2021 and caract_2022

In [ ]:
carateristiques = ps.concat([cleaned_caract_21, cleaned_cara_22])
carateristiques.info()

In [ ]:
clean_carart = carateristiques.astype(caract_types)
clean_carart.dropna()

In [ ]:
clean_carart.info()

In [ ]:
clean_carart.to_parquet('caracteristiques.parquet')

In [ ]:
clean_carart.tail(30)

In [ ]:
clean_carart.head(30)

# Data cleaning du fichier lieu_2021.csv

In [ ]:
file_path = os.path.join(path_data, lieux_2021_file)

sdf_l_21 = ps.read_csv(file_path, sep=';')

bilan:
- type des colonnes
- drop column V2 , lartpc and larrout ,pr et pr1
- drop les na 
- ajout de la colonne annee
- format de la colonne annee en date time 

In [ ]:
sdf_l_21.info()

In [ ]:
sdf_l_21.head(30)

In [ ]:
sdf_l_21.isnull().sum()

In [ ]:
sdf_l_21['larrout'].unique()

In [ ]:
sdf_l_21.isna().sum()

In [ ]:
sdf_l_21['v2'].unique()

In [ ]:
sdf_l_21['lartpc'].unique()

drop les colonnes 
- V2
- lartpc
- larrout  
- pr  
- pr1 

In [ ]:
sdf_l_21 = sdf_l_21.drop(columns=['lartpc'])
sdf_l_21 = sdf_l_21.drop(columns=['v2'])


In [ ]:
sdf_l_21 = sdf_l_21.drop(columns=['larrout'])
sdf_l_21 = sdf_l_21.drop(columns=['pr'])
sdf_l_21 = sdf_l_21.drop(columns=['pr1'])

In [ ]:
sdf_l_21.info()


In [ ]:
sdf_l_21.isnull().sum()

type pour les fichies lieu_2021 et lieu_2022

** categotical columns :
- catr
- circ
- vosp
- prof
- plan
- surf
- infra
- situ
  
  

**numerical columns :
- nbv
- pr1
- larrout
- vma

** continuos columns:
- Num_Acc
- v1
- pr
- voie

**deleted columns



In [ ]:
lieu_types ={
    'Num_Acc': str,
    'catr': np.int64,
    'voie': str,
    'v1': str,
    'circ': np.int64,
    'nbv': np.int64,
    'vosp': np.int64,
    'prof': np.int64,
    'plan': np.int64,
    'surf': np.int64,
    'infra': np.int64,
    'situ': np.int64,
    'vma': np.int64 
}

In [ ]:
clean_lieux_21 = sdf_l_21.astype(lieu_types)

In [ ]:
clean_lieux_21.info()

In [ ]:
clean_lieux_21.isnull().sum()

In [ ]:
clean_lieux_21 = clean_lieux_21.dropna()

In [ ]:
clean_lieux_21.isnull().sum()

In [ ]:
clean_lieux_21['annee'] = 2021

In [ ]:
clean_lieux_21.info()

In [ ]:
# clean_lieux_21['annee'] = ps.to_datetime(clean_lieux_21['annee'], format='%Y')

In [ ]:
#clean_lieux_21.to_parquet('lieux_2021.parquet')

In [ ]:
#la fonction de clean pour les lieu
def clean_lieux_df(sdf, year):
    # drop les column lartpc et v2 , larrout , pr , pr1
    sdf = sdf.drop(columns=['lartpc'])
    sdf = sdf.drop(columns=['v2'])
    sdf = sdf.drop(columns=['larrout'])
    sdf = sdf.drop(columns=['pr'])
    sdf = sdf.drop(columns=['pr1'])
    
    #drop the rows with missing values
    sdf = sdf.dropna()
    
    # covert the type 
    sdf = sdf.astype(lieu_types)
    
    
    # ajout de l'année
    sdf['annee'] = year
    
    # ajout de type annee
    # sdf['annee'] = ps.to_datetime(sdf['annee'], format='%Y')
    return sdf

# Data cleaning du fichier lieu2022


In [ ]:
file_path = os.path.join(path_data, lieux_2022_file)
sdf_l_22 = ps.read_csv(file_path, sep=';')


In [ ]:
sdf_l_22.columns


In [ ]:
sdf_l_22.info()

In [ ]:
clean_lieux_22 = clean_lieux_df(sdf_l_22,2022)

In [ ]:
clean_lieux_22.info()

In [ ]:
#clean_lieux_22.to_parquet('lieux_2022.parquet')

# joining the file lieux 2021 and 2022

In [ ]:
lieu = ps.concat([clean_lieux_21,clean_lieux_22])

lieu.info()

In [ ]:
lieu.head(20)

In [ ]:
lieu.tail(20)

In [ ]:

lieu.isnull().sum()

In [ ]:
lieu = lieu.dropna()

In [ ]:

lieu = lieu.astype(lieu_types)

In [ ]:
lieu.info()

In [ ]:
lieu.to_parquet('lieu.parquet')

# Reading parquet files

In [ ]:
import numpy as np
import pyspark.pandas as ps
import os
import pyarrow.parquet as pq
import plotly.express as px
import plotly.graph_objects as go
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")



reading files from parquet format



In [ ]:
df_v = ps.read_parquet(os.path.join("vehicule_cleaned.parquet"),dtype=vehicule_dtypes)
df_v =df_v.astype(vehicule_dtypes)

df_u = ps.read_parquet(os.path.join("usager_cleaned.parquet"),dtype=usager_dtypes)
df_u =df_u.astype(usager_dtypes)

df_carac = ps.read_parquet(os.path.join("caracteristiques.parquet"))
df_carac =df_carac.astype(caract_types)

df_lieu = ps.read_parquet(os.path.join("lieu.parquet"))
df_lieu =df_lieu.astype(lieu_types)


## Numeric Resume

Let's add the column age for some visualisation after

In [ ]:
df_u["age"] = df_u["annee"] - df_u["an_nais"]
age_col = df_u["age"]
age_col.describe()

Let's caluculate Interquartile difference

In [ ]:
q1 = age_col.quantile(0.25)
q3 = age_col.quantile(0.75)
iqr = q3 - q1
print(iqr)


Now the skewness and kurtosis

In [ ]:
skewness = age_col.skew()
kurtosis = age_col.kurtosis()

print("\nAsymétrie (skewness) :")
print(skewness)


In [ ]:
print("\nAplatissement (kurtosis) :")
print(kurtosis)


In [ ]:


fig = px.box(age_col.to_pandas(), y="age", title=f"Boxplot de l'age ")
fig.show()

We can clearly see the distribution of age and all the quantille

# Question : Repartition of accident

Let's show the repartion of accidents per month

In [ ]:
correspondance_mois = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}
df_carac['nom_mois'] = df_carac['mois'].map(correspondance_mois)
fig = px.histogram(df_carac, x='nom_mois', 
title='Répartition des accidents par mois de l\'année',
labels={'nom_mois': 'Mois de l\'année', 'count': 'Nombre d\'accidents'})
fig.show()

# Usager Profiles

Let's merge the data we need

In [ ]:
merged_usager_carac = ps.merge(df_u, df_carac, on =["Num_Acc"])
merged_usager_carac = merged_usager_carac.astype({**usager_dtypes,**caract_types})
#merged_usager_carac.info()

In [ ]:
merged_usager_lieu = ps.merge(df_u, df_lieu, on =["Num_Acc"])
#merged_usager_lieu = merged_usager_lieu.astype({**lieu_types,**usager_dtypes})
# merged_usager_lieu.info()


In [ ]:
merged_usager_veh = ps.merge(df_u, df_v, on =["Num_Acc","id_vehicule","annee"])
merged_usager_veh = merged_usager_veh.astype({**vehicule_dtypes,**usager_dtypes})
# merged_usager_veh.info()

Let's show  " Distribution de la gravité des accidents par catégorie de route "

In [ ]:
catr_names = {
    1: 'Autoroute',
    2: 'Route nationale',
    3: 'Route départementale',
    4: 'Voie communale',
    5: 'Hors réseau public',
    6: 'Parc de stationnement ouvert à la circulation publique',
    7: 'Routes de métropole urbaine',
    9: 'Autre'
}

grav_names = {
    1: 'Indemne',
    2: 'Tué',
    3: 'Blessé hospitalisé',
    4: 'Blessé léger'
}
merged_usager_lieu['grav_name'] = merged_usager_lieu['grav'].map(grav_names)
merged_usager_lieu['catr_name'] = merged_usager_lieu['catr'].map(catr_names)
grouped_data = merged_usager_lieu.groupby(['catr_name', 'grav_name']).size().reset_index(name='count')
fig_1 = px.bar(grouped_data, x='catr_name', y='count', color='grav_name',
title='Distribution de la gravité des blessures d\'accidents par catégorie de route',
labels={'catr_name': 'Profil de la route', 'count': 'Nombre d\'usagers', 'grav_name': 'Gravité Blessure'},
barmode='group')

fig_1.show()


We can see there are actually less deadly accidents on " Autoroute " roads than people might think. 

In [ ]:
catv_names = {
    # 0: "Indéterminable",
    1: "Bicyclette",
    2: "Cyclomoteur <50cm3",
    # 3: "Voiturette (Quadricycle à moteur carrossé)",
    # 4: "Référence inutilisée depuis 2006 (scooter immatriculé)",
    # 5: "Référence inutilisée depuis 2006 (motocyclette)",
    # 6: "Référence inutilisée depuis 2006 (side-car)",
    7: "VL seul",
    # 8: "Référence inutilisée depuis 2006 (VL + caravane)",
    # 9: "Référence inutilisée depuis 2006 (VL + remorque)",
    # 10: "VU seul 1,5T <= PTAC <= 3,5T avec ou sans remorque",
    # 11: "Référence inutilisée depuis 2006 (VU (10) + caravane)",
    # 12: "Référence inutilisée depuis 2006 (VU (10) + remorque)",
    
    # 13: "PL seul 3,5T <PTCA <= 7,5T",
    # 14: "PL seul > 7,5T",
    # 15: "PL > 3,5T + remorque",
    
    # 16: "Tracteur routier seul",
    # 17: "Tracteur routier + semi-remorque",
    # 18: "Référence inutilisée depuis 2006 (transport en commun)",
    # 19: "Référence inutilisée depuis 2006 (tramway)",
    # 20: "Engin spécial",
    # 21: "Tracteur agricole",
    30: "Scooter < 50 cm3",
    31: "Motocyclette > 50 cm3 et <= 125 cm3",
    32: "Scooter > 50 cm3 et <= 125 cm3",
    33: "Motocyclette > 125 cm3",
    34: "Scooter > 125 cm3",
    # 35: "Quad léger <= 50 cm3 (Quadricycle à moteur non carrossé)",
    # 36: "Quad lourd > 50 cm3 (Quadricycle à moteur non carrossé)",
    37: "Autobus",
    # 38: "Autocar",
    # 39: "Train",
    # 40: "Tramway",
    # 41: "3RM <= 50 cm3",
    # 42: "3RM > 50 cm3 <= 125 cm3",
    # 43: "3RM >"
}

merged_usager_veh['catv_name'] = merged_usager_veh['catv'].map(catv_names)
merged_usager_veh['grav_name'] = merged_usager_veh['grav'].map(grav_names)

grouped_data = merged_usager_veh.groupby(['catv_name', 'grav_name']).size().reset_index(name='count')

fig_2 = px.bar(grouped_data, x='catv_name', y='count', color='grav_name',
title='Distribution de la gravité des blessures d\'accidents par rapport au vehicule impliqué',
labels={'catv_name': 'Catégorie de vehicule', 'count': 'Nombre d\'usagers', 'grav_name': 'Gravité Blessure'},
barmode='group')

fig_2.show()

The " Vl seul" actually has more accidents than all the other types of vehicules combined. That type of vehicule might be the problem, who knows.

As you guessed, man are the most involved in accidents

In [ ]:
prof_names = {
    # -1: 'Non Renseigné',
    1: 'Plat',
    2: 'Pente',
    3: 'Sommet de côte',
    4: 'Bas de côte'
}
sexe_names = {
    1: 'Masculin',
    2: 'Feminin'
}

merged_usager_lieu['prof_name'] = merged_usager_lieu['prof'].map(prof_names)
merged_usager_lieu['sexe_name'] = merged_usager_lieu['sexe'].map(sexe_names)

grouped_data = merged_usager_lieu.groupby(['prof_name', 'sexe_name']).size().reset_index(name='count')

fig_3 = px.bar(grouped_data, x='prof_name', y='count', color='sexe_name',
title='Distribution des sexe par type du lieu d\'accident',
labels={'prof_name': 'Type de la route', 'count': 'Nombre d\'usagers', 'sexe_name': 'Sexe'},
barmode='group')

fig_3.show()

As you guessed, men are more involved in accidents than women on any type of roads

In [ ]:

catu_names = {
    1: 'Conducteur',
    2: 'Passager',
    3: 'Piéton'
}
# catu_values = merged_usager_lieu['catu'].cat.categories
# mapped_values = [catu_names[value] for value in catu_values]
# mapping = dict(zip(catu_values, mapped_values))
# merged_usager_lieu['catu_name'] = merged_usager_lieu['catu'].map(mapping)
merged_usager_lieu['catu_name'] = merged_usager_lieu['catu'].map(catu_names)
grouped_data = merged_usager_lieu.groupby(['catu_name', 'grav_name']).size().reset_index(name='count')

fig_4 = px.bar(grouped_data, x='catu_name', y='count', color='grav_name',
title='Distribution des usagers par rapport à la gravité de leur accidents',
labels={'catu_name': 'Catégorie d\'usager', 'count': 'Nombre d\'usagers', 'grav_name': 'Gravité Accident'},
barmode='group')

# Afficher le graphique
fig_4.show()

# Question : " Usage des types composites "

Let's merge the data that we need

In [ ]:
merged_veh_lieu = ps.merge(df_v, df_lieu, on =["Num_Acc"])
merged_veh_lieu = merged_veh_lieu.astype({**vehicule_dtypes,**lieu_types})
merged_veh_lieu['categorie_route'] = merged_veh_lieu['catr'].map(catr_names)

# merged_veh_lieu.info()

We select only the columns that we need and group it

In [ ]:

from pyspark.sql.functions import collect_list, struct
df = merged_veh_lieu[["Num_Acc","categorie_route","id_vehicule"]]
df = df.to_spark()
grouped_df = df.groupby("Num_Acc", "categorie_route") \
    .agg(collect_list(struct("id_vehicule")).alias("id_vehicule_list"))

In [ ]:

grouped_df.show(truncate=False)

The result is exactely what we wanted

# Stats resume with plotly for carateristique: 
- describe 
- skewness of each column
- kurtosis of each column
- box plot of each column
- column correlation
- histogram
  

statitique descriptive de chaque colonne

In [ ]:
df_carac.describe()

our remarks are:
- most accident occured during the day
- most accident occured in agglomerations (cities, towns etc)
- most accident occured during normal atmospheric conditions

skewness and kurtosis of each column


In [ ]:
print("--------------skewness------------------------")
skewness_caract = df_carac.skew()
print(skewness_caract)
print("--------------kurtosis------------------------")
kurto_caract = df_carac.kurtosis()
print(kurto_caract)

verifions avec des histogram


let see the box plot 

1)  the box of accident that occured during the day through the years

In [ ]:
# lumiere : conditions de luminosité de l'accident
lumiere = { 
    1: 'Plein jour',
    2: 'Crépuscule ou aube',
    3: 'Nuit sans éclairage public',
    4: 'Nuit avec éclairage public non allumé',
    5: 'Nuit avec éclairage public allumé'
}
df_carac['periode_lum'] = df_carac['lum'].map(lumiere)

In [ ]:
#atm : conditions atmosphériques
atmosphere = {
    1: 'Normale',
    2: 'Pluie légère',
    3: 'Pluie forte',
    4: 'Neige - grêle',
    5: 'Brouillard - fumée',
    6: 'Vent fort - tempête',
    7: 'Temps éblouissant',
    8: 'Temps couvert',
    9: 'Autre'
}
df_carac['conditions_atmos'] = df_carac['atm'].map(atmosphere)

In [ ]:
#agg : agglomération
agglomeration = {
    1: 'Hors agglomération',
    2: 'En agglomération'
}
df_carac['agglomeration'] = df_carac['agg'].map(agglomeration)

In [ ]:
fig = px.box(df_carac, x='an', y='Num_Acc')
fig.show()

In [ ]:
fig = px.box(df_carac, x='an', y='periode_lum', color='an', title='repartition de la periode des accidents par année')
fig.show()

pas tres convaincant

histogram 

In [ ]:
fig = px.histogram(df_carac, x='periode_lum', 
title='Répartition des accidents par intensité durant  l\'année',
labels={'periode_lum': 'l\'année', 'count': 'Nombre d\'accidents'}, color='an')
fig.show()

In [ ]:
fig = px.histogram(df_carac, x='conditions_atmos',
title='Répartition des accidents par conditions atmosphériques',
labels={'conditions_atmos': 'Conditions atmosphériques', 'count': 'Nombre d\'accidents'}, color='conditions_atmos')
fig.show()

In [ ]:
fig = px.histogram(df_carac, x='agglomeration',
title='Répartition des accidents par agglomération(ville ,citée, village, etc...)',
labels={'agglomeration': 'Agglomération', 'count': 'Nombre d\'accidents'}, color='agglomeration')
fig.show()

les repartion des accidents par mois, an 

In [ ]:
fig = px.histogram(df_carac, x="an", nbins=2, title="Répartition des accidents par année", color="an" , labels={'an': 'Année', 'count': 'Nombre d\'accidents'})
fig.show()

fig = px.histogram(df_carac, x="nom_mois", nbins=12, title="Répartition des accidents par mois", color="mois" , labels={'nom_mois': 'Mois de l\'année', 'count': 'Nombre d\'accidents'})
fig.show()

# pas tres pertinent ces 2
fig = px.histogram(df_carac, x="jour", nbins=31, title="Répartition des accidents par jour", color="jour")
fig.show()

fig = px.pie(df_carac, values="Num_Acc", names="jour", title="Répartition des accidents par jour", color="jour")
fig.show()

stat resume with plotly for lieux:

In [ ]:
df_lieu.describe()

en moyenne les accidents sont en vitesse maximale 58 km/h

regardons les histograms

In [ ]:
#situation : situation de l'accident
situation = {
    1: 'Sur chaussée',
    2: 'Sur bande d\'arrêt d\'urgence',
    3: 'Sur accotement',
    4: 'Sur trottoir',
    5: 'Sur piste cyclable',
    6: 'Sur autre voie spéciale',
    9: 'Autre'
}
df_lieu['situation_accident'] = df_lieu['situ'].map(situation)

In [ ]:
fig = px.histogram(df_lieu, x='situation_accident',
title='Répartition des accidents par situation de l\'accident',
labels={'situation_accident': 'Situation de l\'accident', 'count': 'Nombre d\'accidents'}, color='situation_accident')
fig.show()